<a href="https://colab.research.google.com/github/yfqian95/JWAS.jl/blob/master/Aug3_Bayesian_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia CSV DataFrames Random Statistics Distributions LinearAlgebra SparseArrays Printf JWAS"  # CSV DataFrames Random Statistics JWAS Distributions LinearAlgebra
JULIA_PACKAGES_IF_GPU=""
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-08-03 16:36:43 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package CSV...
Installing Julia package DataFrames...
Installing Julia package Random...
Installing Julia package Statistics...
Installing Julia package Distributions...
Installing Julia package LinearAlgebra...
Installing Julia package SparseArrays...
Installing Julia package Printf...
Installing Julia package JWAS...


In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


In [6]:
using Pkg
Pkg.add("Plots")
using Plots,Statistics,DataFrames, Random

In [7]:
Random.seed!(123)
mean_male   = 170
sd_male     = 5
mean_female = 159
sd_female   = 5
nind_male   = 1200
nind_female = 800
males       = mean_male .+ randn(nind_male)*sd_male
females     = mean_female .+ randn(nind_female)*sd_female
data=[convert.(Int,round.(males))  fill("male",nind_male)
      convert.(Int,round.(females)) fill("female",nind_female)];
data = DataFrame(height=data[:,1],gender=data[:,2])
first(data,5)

,height,gender
,Any,Any
1,174,male
2,164,male
3,164,male
4,168,male
5,171,male


In [ ]:
plot(data[data[!,:gender].=="male",:height],seriestype=:histogram,xlabel=:height,legend=:none) 
plot!(data[data[!,:gender].=="female",:height],seriestype=:histogram) 

In [9]:
using Distributions
μ = [1.0;2.0]
V = [1.0 0.5;0.5 2.0] #V is variance for each variable

x     = [0.0;0.0] #initial value
niter = 50000 #number of iteration
c     = 0 #number of times that the condition happens 

for i in 1:niter
    mean12 = μ[1]+V[1,2]/V[2,2]*(x[2]-μ[2])
    var12  = V[1,1]-V[1,2]^2/V[2,2]
    
    x[1] = mean12+randn()*sqrt(var12)
    
    mean21 = μ[2]+V[2,1]/V[1,1]*(x[1]-μ[1])
    var21  = V[2,2]-V[2,1]^2/V[1,1]
    
    x[2] = mean21+randn()*sqrt(var21)
    
    if x[1] > 1 && x[2] > 2
        c += 1
    end
end

c/niter

0.3087

In [10]:
using Distributions
μ = [1;2]
V = [1.0 0.5;0.5 2.0]

x     = [0;0]
niter = 50000
c     = 0

for i in 1:niter
    x = rand(MvNormal(μ, V))
    if x[1] > 1 && x[2] > 2
        c += 1
    end
end

c/niter

0.30886